# Capstone Project - Aggregate scoring

__Recommendation for running shoes based on:__
- [Closest aggregate score based on similarity of shoe features, ratings, reviews](#shortcut1)

In [1]:
import pickle

import numpy as np
import pandas as pd
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 30)
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

# gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim import similarities

### Load in data

#### Overall shoe data

In [2]:
# Load overall shoe data
shoe_data = pickle.load(open('shoe_data.pkl', 'rb'))
print(shoe_data.shape)
shoe_data.head()

(848, 49)


,name,brand,core_score,user_review_count,expert_review_count,user_rating,expert_rating,user_5star,user_4star,user_3star,user_2star,user_1star,discontinued,terrain,arch_support,...,upper,weight_m,weight_w,toe_drop_m,toe_drop_w,heel_height_m,heel_height_w,forefoot_height_m,forefoot_height_w,width_m,width_w,zero_drop_m,zero_drop_w,low_drop_m,low_drop_w
0,Nike Air Zoom Pegasus 35,Nike,94,2249,27,4.4,94.0,71,14,6,4,5,1,1,Neutral,...,The upper unit of the Nike Air Zoom Pegasus 35...,286,255,10,10,29,29,19,19,"Narrow, Standard, Wide, Extra wide","Standard, Wide",0,0,0,0
1,Brooks Ghost 11,Brooks,94,4358,17,4.5,91.0,78,9,5,3,5,1,1,Neutral,...,The engineered mesh upper of the Brooks Ghost ...,309,269,12,12,29,29,17,17,"Narrow, Standard, Wide, Extra wide","Narrow, Standard, Wide",0,0,0,0
2,Asics Gel Kayano 25,Asics,95,4091,8,4.6,92.0,79,11,3,2,5,1,1,Stability,...,The Gel Kayano 25 welcomes the use of Jacquard...,336,278,10,13,22,24,12,11,"Narrow, Standard, Wide, Extra wide","Narrow, Standard, Wide",0,0,0,0
3,Asics Gel Venture 6,Asics,94,4158,1,4.5,90.0,74,14,5,3,4,1,0,Neutral,...,The synthetic mesh material covers the upper s...,318,255,10,10,20,20,10,10,"Standard, Wide, Extra wide","Narrow, Standard, Wide, Extra wide",0,0,0,0
4,Hoka One One Bondi 6,Hoka One One,91,794,7,4.2,92.0,64,14,9,7,6,0,1,Neutral,...,The upper unit of the Hoka One One Bondi 6 mak...,309,244,4,4,36,36,32,32,"Narrow, Standard, Wide","Narrow, Standard, Wide",0,0,1,1


#### TF-IDF similarity scores

In [3]:
# Load pickle files for TF-IDF
version = 'v8-386-bigram'
to_unpickle = [
    'df_sims_good_summary',
    'corpus_good_summary',
    'tfidf_good_summary',
    'dictionary_good_summary',
    'df_sims_info',
    'corpus_info',
    'tfidf_info',
    'dictionary_info',
    'df_sims_size_fit',
    'corpus_size_fit',
    'tfidf_size_fit',
    'dictionary_size_fit',
    'df_sims_outsole',
    'corpus_outsole',
    'tfidf_outsole',
    'dictionary_outsole',
    'df_sims_midsole',
    'corpus_midsole',
    'tfidf_midsole',
    'dictionary_midsole',
    'df_sims_upper',
    'corpus_upper',
    'tfidf_upper',
    'dictionary_upper',
    'shoes'
]

In [4]:
shoes = pickle.load(open('tfidf_files/shoes_{}.pkl'.format(version), 'rb'))

tfidf_good_summary = pickle.load(open('tfidf_files/tfidf_good_summary_{}.pkl'.format(version), 'rb'))
tfidf_info = pickle.load(open('tfidf_files/tfidf_info_{}.pkl'.format(version), 'rb'))
tfidf_size_fit = pickle.load(open('tfidf_files/tfidf_size_fit_{}.pkl'.format(version), 'rb'))
tfidf_outsole = pickle.load(open('tfidf_files/tfidf_outsole_{}.pkl'.format(version), 'rb'))
tfidf_midsole = pickle.load(open('tfidf_files/tfidf_midsole_{}.pkl'.format(version), 'rb'))
tfidf_upper = pickle.load(open('tfidf_files/tfidf_upper_{}.pkl'.format(version), 'rb'))

corpus_good_summary = pickle.load(open('tfidf_files/corpus_good_summary_{}.pkl'.format(version), 'rb'))
corpus_info = pickle.load(open('tfidf_files/corpus_info_{}.pkl'.format(version), 'rb'))
corpus_size_fit = pickle.load(open('tfidf_files/corpus_size_fit_{}.pkl'.format(version), 'rb'))
corpus_outsole = pickle.load(open('tfidf_files/corpus_outsole_{}.pkl'.format(version), 'rb'))
corpus_midsole = pickle.load(open('tfidf_files/corpus_midsole_{}.pkl'.format(version), 'rb'))
corpus_upper = pickle.load(open('tfidf_files/corpus_upper_{}.pkl'.format(version), 'rb'))

dictionary_good_summary = pickle.load(open('tfidf_files/dictionary_good_summary_{}.pkl'.format(version), 'rb'))
dictionary_info = pickle.load(open('tfidf_files/dictionary_info_{}.pkl'.format(version), 'rb'))
dictionary_size_fit = pickle.load(open('tfidf_files/dictionary_size_fit_{}.pkl'.format(version), 'rb'))
dictionary_outsole = pickle.load(open('tfidf_files/dictionary_outsole_{}.pkl'.format(version), 'rb'))
dictionary_midsole = pickle.load(open('tfidf_files/dictionary_midsole_{}.pkl'.format(version), 'rb'))
dictionary_upper = pickle.load(open('tfidf_files/dictionary_upper_{}.pkl'.format(version), 'rb'))

tfidf_sims_good_summary = pickle.load(open('tfidf_files/df_sims_good_summary_{}.pkl'.format(version), 'rb'))
tfidf_sims_info = pickle.load(open('tfidf_files/df_sims_info_{}.pkl'.format(version), 'rb'))
tfidf_sims_size_fit = pickle.load(open('tfidf_files/df_sims_size_fit_{}.pkl'.format(version), 'rb'))
tfidf_sims_outsole = pickle.load(open('tfidf_files/df_sims_outsole_{}.pkl'.format(version), 'rb'))
tfidf_sims_midsole = pickle.load(open('tfidf_files/df_sims_midsole_{}.pkl'.format(version), 'rb'))
tfidf_sims_upper = pickle.load(open('tfidf_files/df_sims_upper_{}.pkl'.format(version), 'rb'))

In [5]:
print(tfidf_sims_good_summary.shape)
tfidf_sims_good_summary.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.015909,0.028599,0.027614,0.018213,0.018810,0.015021,0.022670,0.025260,0.054418,0.053180,0.011190,0.034852,0.018957,0.018495,...,0.043729,0.017039,0.019962,0.008787,0.009228,0.003600,0.010208,0.013146,0.005060,0.012248,0.005991,0.010521,0.026157,0.007044,0.000484
1,0.015909,1.000000,0.038609,0.009135,0.024169,0.035964,0.005722,0.053422,0.028967,0.031546,0.014808,0.018860,0.002728,0.018692,0.023280,...,0.004929,0.001784,0.000418,0.034760,0.002518,0.008159,0.006032,0.007079,0.002733,0.008738,0.059898,0.004822,0.008347,0.008332,0.006816
2,0.028599,0.038609,1.000000,0.019201,0.014880,0.022604,0.014705,0.043204,0.020710,0.061628,0.025528,0.022819,0.021933,0.041353,0.033994,...,0.055474,0.008060,0.020163,0.010748,0.014043,0.011045,0.002961,0.024864,0.010394,0.013395,0.008166,0.007915,0.025131,0.012471,0.012489


In [6]:
print(tfidf_sims_info.shape)
tfidf_sims_info.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.004407,0.004089,0.002720,0.013689,0.001525,0.004495,0.008936,0.005196,0.012158,0.006302,0.011135,0.038905,0.008876,0.006132,...,0.032337,0.015222,0.009442,0.002604,0.015304,0.042957,0.022885,0.018665,0.003024,0.003911,0.026030,0.023770,0.049540,0.014199,0.013957
1,0.004407,1.000000,0.018523,0.006222,0.004767,0.007400,0.002225,0.078201,0.027152,0.025229,0.153546,0.057650,0.015947,0.004599,0.020336,...,0.010232,0.016327,0.003398,0.008018,0.008658,0.005711,0.005672,0.006594,0.004418,0.012020,0.011447,0.013541,0.008592,0.013264,0.009409
2,0.004089,0.018523,1.000000,0.010072,0.005855,0.003389,0.005486,0.017626,0.011186,0.023344,0.041268,0.018011,0.009485,0.013905,0.021273,...,0.005816,0.003900,0.014497,0.019433,0.005132,0.013277,0.006283,0.002104,0.010552,0.007188,0.018728,0.014094,0.006934,0.009428,0.007139


In [7]:
print(tfidf_sims_size_fit.shape)
tfidf_sims_size_fit.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.105300,0.028689,0.037096,0.117751,0.066367,0.023610,0.297518,0.019079,0.041410,0.073238,0.075256,0.124060,0.059476,0.012693,...,0.060185,0.102946,0.042862,0.013944,0.061136,0.054951,0.072597,0.011063,0.019228,0.004781,0.014712,0.022783,0.094085,0.029993,0.021254
1,0.105300,1.000000,0.129034,0.032395,0.052373,0.004497,0.036154,0.137365,0.004717,0.029834,0.055551,0.057373,0.063110,0.003192,0.037164,...,0.021984,0.002748,0.000814,0.015014,0.010472,0.014589,0.002192,0.010858,0.002229,0.008216,0.014950,0.016048,0.021797,0.013540,0.016087
2,0.028689,0.129034,1.000000,0.037848,0.040773,0.024546,0.002379,0.068535,0.078903,0.029708,0.038977,0.049378,0.054154,0.017423,0.016171,...,0.037870,0.008430,0.003343,0.109297,0.024107,0.012673,0.010665,0.036513,0.018825,0.003267,0.031607,0.034486,0.019604,0.011954,0.048398


In [8]:
print(tfidf_sims_outsole.shape)
tfidf_sims_outsole.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.024899,0.013381,0.005539,0.022786,0.009549,0.079585,0.023789,0.014994,0.018019,0.024777,0.022151,0.033362,0.003801,0.034454,...,0.040059,0.005986,0.034889,0.012717,0.008209,0.017961,0.027109,0.009555,0.009088,0.012680,0.004272,0.015418,0.031659,0.010561,0.016089
1,0.024899,1.000000,0.025191,0.012385,0.001522,0.001695,0.011154,0.015749,0.025692,0.008619,0.037784,0.004411,0.017097,0.003532,0.017930,...,0.032476,0.008423,0.007894,0.015487,0.005852,0.006797,0.004687,0.005203,0.052175,0.020852,0.001859,0.004057,0.025518,0.005091,0.015996
2,0.013381,0.025191,1.000000,0.011476,0.007243,0.023697,0.015375,0.016196,0.030352,0.061306,0.041069,0.007468,0.006867,0.005393,0.011766,...,0.049022,0.004629,0.013646,0.007714,0.005662,0.005022,0.003075,0.005004,0.016729,0.027173,0.006496,0.014879,0.017279,0.043835,0.022650


In [9]:
print(tfidf_sims_midsole.shape)
tfidf_sims_midsole.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.008241,0.003749,0.003098,0.000916,0.004876,0.051327,0.010464,0.011467,0.012447,0.001946,0.008773,0.019730,0.020527,0.000827,...,0.011338,0.002980,0.004829,0.012442,0.018054,0.013581,0.006054,0.003183,0.009622,0.005092,0.004856,0.018482,0.011377,0.027739,0.004679
1,0.008241,1.000000,0.030664,0.017442,0.009402,0.004371,0.010798,0.046609,0.022141,0.020354,0.141485,0.046238,0.017610,0.014102,0.007940,...,0.006253,0.012627,0.014302,0.024807,0.008298,0.008594,0.012831,0.003540,0.003251,0.009228,0.024999,0.006459,0.014745,0.015086,0.007142
2,0.003749,0.030664,1.000000,0.018943,0.011271,0.002646,0.014230,0.039272,0.027668,0.111558,0.002710,0.011674,0.020768,0.009350,0.022225,...,0.010535,0.011748,0.043056,0.019998,0.001893,0.014545,0.029343,0.002217,0.003232,0.017501,0.040582,0.016391,0.014183,0.020801,0.007782


In [10]:
print(tfidf_sims_upper.shape)
tfidf_sims_upper.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.003354,0.014637,0.012760,0.060256,0.003277,0.080964,0.005726,0.019356,0.028041,0.019024,0.043582,0.027604,0.022447,0.011224,...,0.010053,0.039419,0.033863,0.005754,0.008671,0.008476,0.008504,0.030424,0.004560,0.000575,0.005256,0.037519,0.006414,0.024303,0.001507
1,0.003354,1.000000,0.029675,0.008711,0.007958,0.012810,0.004625,0.008439,0.017996,0.026396,0.018396,0.013971,0.006929,0.013425,0.007470,...,0.005452,0.002909,0.003258,0.028075,0.022946,0.005354,0.024355,0.003029,0.006581,0.013499,0.013873,0.007962,0.001001,0.021324,0.009631
2,0.014637,0.029675,1.000000,0.011107,0.017550,0.009857,0.017944,0.007662,0.023948,0.029016,0.028516,0.021413,0.027847,0.029926,0.008164,...,0.004087,0.016135,0.016123,0.018674,0.019017,0.014079,0.012227,0.007342,0.012258,0.011211,0.033492,0.016704,0.009768,0.012842,0.005571


#### Combined similarity matrix for shoes

In [11]:
# Load shoe features similarity matrices
comb_m_sim = pickle.load(open('comb_m_sim.pkl', 'rb'))
comb_w_sim = pickle.load(open('comb_w_sim.pkl', 'rb'))

# brand_matrix = pickle.load(open('brand_matrix.pkl', 'rb'))
# terrain_matrix = pickle.load(open('terrain_matrix.pkl', 'rb'))
# arch_matrix = pickle.load(open('arch_matrix.pkl', 'rb'))
# use_matrix = pickle.load(open('use_matrix.pkl', 'rb'))

In [12]:
print(comb_m_sim.shape)
comb_m_sim.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.999156,0.732149,0.839703,0.981959,0.661857,0.999832,0.750142,0.988845,0.988934,0.995844,0.995850,0.995710,0.852569,0.861334,...,0.993881,0.833965,0.998267,0.864057,0.848630,0.751838,0.859888,0.996800,0.990266,0.866514,0.849118,0.840882,0.847789,0.994929,0.824596
1,0.999156,1.000000,0.736185,0.845096,0.976628,0.663318,0.999335,0.750106,0.988367,0.991681,0.994223,0.994197,0.991585,0.849952,0.859000,...,0.991193,0.832592,0.997179,0.860771,0.850162,0.748496,0.858559,0.996264,0.991255,0.869641,0.852601,0.832050,0.838917,0.992548,0.821940
2,0.732149,0.736185,1.000000,0.516283,0.726957,0.213595,0.732099,0.732731,0.746864,0.743948,0.741960,0.741825,0.726809,0.520965,0.558232,...,0.721953,0.486509,0.738012,0.548012,0.586020,0.965351,0.535087,0.727935,0.746606,0.647079,0.526495,0.520760,0.512229,0.710508,0.472862


In [13]:
print(comb_w_sim.shape)
comb_w_sim.head(3)

(848, 848)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847
0,1.000000,0.999466,0.731780,0.841470,0.982764,0.654711,0.999488,0.748249,0.986164,0.990137,0.995907,0.995910,0.995262,0.853392,0.859707,...,0.993017,0.835401,0.996787,0.863065,0.848767,0.748599,0.860624,0.994885,0.991526,0.861226,0.847927,0.842680,0.853868,0.994553,0.824677
1,0.999466,1.000000,0.734001,0.843951,0.977604,0.652671,0.998394,0.747410,0.984943,0.992396,0.994578,0.994466,0.991693,0.851577,0.857219,...,0.990694,0.834794,0.996359,0.859362,0.848290,0.744334,0.860711,0.993872,0.991548,0.864229,0.849177,0.834619,0.847542,0.992842,0.824096
2,0.731780,0.734001,1.000000,0.502529,0.725335,0.185811,0.728635,0.731598,0.739544,0.740493,0.741762,0.741217,0.725517,0.523699,0.555711,...,0.717870,0.489460,0.738192,0.545089,0.576836,0.965590,0.540333,0.720827,0.740306,0.639128,0.513623,0.522325,0.524164,0.711561,0.477280


#### User ratings & popularity score

In [14]:
# Load user ratings & crack score
ratings = pickle.load(open('ratings.pkl', 'rb'))

print(ratings.shape)
ratings.head(3)

(848, 8)


,user_rating_sc,expert_rating_sc,average_rating_sc,user_reviews,expert_reviews,user_popularity,expert_popularity,average_popularity_sc
0,0.88,0.94,0.910,2249,27,0.680211,0.838246,0.759228
1,0.90,0.91,0.905,4358,17,0.753871,0.720585,0.737228
2,0.92,0.92,0.920,4091,8,0.746831,0.528874,0.637853


### Combine into Scoring Matrix

In [15]:
reccs = {}
for i in shoe_data.index:
    # Combine all the scores into one matrix.
    comb = pd.concat([
        shoe_data.name,
        tfidf_sims_good_summary[i],
        tfidf_sims_info[i],
        tfidf_sims_size_fit[i],
        tfidf_sims_outsole[i],
        tfidf_sims_midsole[i],
        tfidf_sims_upper[i],
        comb_m_sim[i],
        ratings[['average_rating_sc', 'average_popularity_sc']]
    ], axis=1)
    comb.columns = [
        'name',
        'good_summary',
        'info',
        'size_fit',
        'outsole',
        'midsole',
        'upper',
        'features',
        'avg_rating',
        'avg_popularity'
    ]
    
    # Drop the shoe which the recommendations will be found for.
    comb.drop(i, axis=0, inplace=True)

    # Rescale and add score weightings.
    mm = MinMaxScaler()
    comb.good_summary = mm.fit_transform(comb.good_summary.values.reshape(-1, 1))
    comb['info'] = mm.fit_transform(comb['info'].values.reshape(-1, 1))
    comb.size_fit = mm.fit_transform(comb.size_fit.values.reshape(-1, 1))
    comb.outsole = mm.fit_transform(comb.outsole.values.reshape(-1, 1))
    comb.midsole = mm.fit_transform(comb.midsole.values.reshape(-1, 1))
    comb.upper = mm.fit_transform(comb.upper.values.reshape(-1, 1))
    comb.features = mm.fit_transform(comb.features.values.reshape(-1, 1))
    comb.avg_rating = mm.fit_transform(comb.avg_rating.values.reshape(-1, 1)) * 0.7
    comb.avg_popularity = mm.fit_transform(comb.avg_popularity.values.reshape(-1, 1)) * 0.7

#     # Rename columns.
#     comb.columns = [
#         'Name',
#         'Theme',
#         'Features',
#         'High user rating',
#         'Popularity'
#     ]

    # Calculation of aggregate score.
    best = comb.copy()
    best['aggregate'] = comb.mean(axis=1, skipna=True)
    
    # Extract shoes of greatest similarity, put into top of DataFrame.
    top = best.sort_values('aggregate', ascending=False).head(10)
    
    # Add into dictionary.
    reccs[i] = top

In [16]:
len(reccs)

848

<a id='shortcut1'></a>
### Get recommendations for men's running shoes

In [17]:
shoe_data.index[shoe_data['name'] == "Brooks Glycerin 17"]

Int64Index([11], dtype='int64')

In [18]:
# Get recommendation for shoe using its index.
shoe_idx = 11
print(f"Shoe name: {shoe_data.name[shoe_idx]}")
print("Most similar shoe has the highest aggregate score.")
reccs[shoe_idx]

Shoe name: Brooks Glycerin 17
Most similar shoe has the highest aggregate score.


,name,good_summary,info,size_fit,outsole,midsole,upper,features,avg_rating,avg_popularity,aggregate
10,Brooks Glycerin 16,0.108523,1.000000,0.914153,0.010847,0.737712,0.575652,1.000000,0.525000,0.509901,0.597977
112,Brooks Ghost 12,0.207466,0.233175,0.549399,0.528930,0.347448,0.823415,0.997673,0.525000,0.268642,0.497905
16,Adidas Ultraboost 19,0.651056,0.115664,0.847992,0.430587,0.226244,0.288395,0.991707,0.525000,0.300109,0.486306
12,Hoka One One Clifton 5,0.183286,0.220603,0.514470,0.515139,0.342368,0.420811,0.994249,0.525000,0.568570,0.476055
114,Salomon Sense Ride 2,0.582018,0.146679,0.534560,0.316962,0.300284,0.724636,0.780446,0.560000,0.331251,0.475204
54,Adidas Solar Glide,1.000000,0.296170,0.134567,0.313128,0.114623,0.515535,0.995325,0.560000,0.294745,0.469344
15,Adidas Pure Boost Go,0.585687,0.276595,0.757562,0.439227,0.020633,0.308943,0.967927,0.536667,0.316777,0.467780
285,Topo Athletic Terraventure 2,0.267969,0.657955,0.650943,0.171654,0.521809,0.328793,0.773327,0.548333,0.264941,0.465080
99,Asics DynaFlyte 3,0.265545,0.398879,0.226089,1.000000,0.156732,0.421395,0.974552,0.501667,0.210364,0.461691
113,Saucony Ride 10,0.229971,0.313820,0.746356,0.189238,0.190874,0.568457,0.988698,0.478333,0.408186,0.457104


In [19]:
test_m = reccs[shoe_idx].sort_values(by='aggregate', ascending=False).head()
test_m_index_list = test_m.index.to_list()

rec_4 = shoe_data.iloc[test_m_index_list, [0,1,13,14,15,16,17,18,19,20,21,35,37,39,41,43,45,47]]
rec_4 = pd.concat([rec_4, test_m.drop('name', axis=1)], axis=1)
rec_4.to_csv('rec_4.csv', index=False)
rec_4

,name,brand,terrain,arch_support,use,waterproof,water_repellent,maximalist,minimalist,triathlon,price,weight_m,toe_drop_m,heel_height_m,forefoot_height_m,width_m,zero_drop_m,low_drop_m,good_summary,info,size_fit,outsole,midsole,upper,features,avg_rating,avg_popularity,aggregate
10,Brooks Glycerin 16,Brooks,1,Neutral,1,0,0,0,0,0,290,301,10,32,22,"Standard, Wide",0,0,0.108523,1.000000,0.914153,0.010847,0.737712,0.575652,1.000000,0.525,0.509901,0.597977
112,Brooks Ghost 12,Brooks,1,Neutral,1,0,0,0,0,0,250,295,10,32,22,Standard,0,0,0.207466,0.233175,0.549399,0.528930,0.347448,0.823415,0.997673,0.525,0.268642,0.497905
16,Adidas Ultraboost 19,Adidas,1,Neutral,1,0,0,0,0,0,350,309,10,29,19,Standard,0,0,0.651056,0.115664,0.847992,0.430587,0.226244,0.288395,0.991707,0.525,0.300109,0.486306
12,Hoka One One Clifton 5,Hoka One One,1,Neutral,1,0,0,1,0,0,250,266,5,30,25,"Standard, Wide",0,0,0.183286,0.220603,0.514470,0.515139,0.342368,0.420811,0.994249,0.525,0.568570,0.476055
114,Salomon Sense Ride 2,Salomon,0,Neutral,1,0,0,0,0,0,230,270,7,25,18,Standard,0,0,0.582018,0.146679,0.534560,0.316962,0.300284,0.724636,0.780446,0.560,0.331251,0.475204
